<a href="https://colab.research.google.com/github/joel280584/PineConeLearning/blob/main/Query_IMDB_DB_for_movie_list.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Query IMDB DB for movie list

In [19]:
!pip install pinecone
!pip install -U sentence-transformers

In [20]:
from google.colab import userdata
apikey = userdata.get('API_KEY')
from pinecone import Pinecone, ServerlessSpec
pc= Pinecone(api_key=apikey)

In [ ]:
pc.list_indexes()
pc.delete_index("quickstart")
pc.list_indexes()

[]

In [ ]:
index_name = "quickstart"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [21]:
#pc.delete_index(index_name)
pc.list_indexes()

[
    {
        "name": "quickstart",
        "dimension": 384,
        "metric": "cosine",
        "host": "quickstart-2s9kc7h.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "deletion_protection": "disabled"
    }
]

In [22]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
imdb_df = pd.read_csv('/content/drive/MyDrive/Untitled folder/imdb_movies.csv')
imdb_df.head()

,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
0,Creed III,03/02/2023,73.0,"Drama, Action","After dominating the boxing world, Adonis Cree...","Michael B. Jordan, Adonis Creed, Tessa Thompso...",Creed III,Released,English,75000000.0,2.716167e+08,AU
1,Avatar: The Way of Water,12/15/2022,78.0,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,"Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Avatar: The Way of Water,Released,English,460000000.0,2.316795e+09,AU
2,The Super Mario Bros. Movie,04/05/2023,76.0,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...","Chris Pratt, Mario (voice), Anya Taylor-Joy, P...",The Super Mario Bros. Movie,Released,English,100000000.0,7.244590e+08,AU
3,Mummies,01/05/2023,70.0,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three ...","Óscar Barberán, Thut (voice), Ana Esther Albor...",Momias,Released,"Spanish, Castilian",12300000.0,3.420000e+07,AU
4,Supercell,03/17/2023,61.0,Action,Good-hearted teenager William always lived in ...,"Skeet Ulrich, Roy Cameron, Anne Heche, Dr Quin...",Supercell,Released,English,77000000.0,3.409420e+08,US


In [24]:
imdb_df.shape

(10178, 12)

In [25]:
imdb_df.isna().sum()
imdb_df.reset_index(drop=True)
imdb_df.shape
imdb_df.iloc[[10051]]


,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
10051,Cabin Fever,02/12/2016,47.0,Horror,"In this grisly remake of the 2002 horror hit, ...","Eli Roth, , Teresa Decher, Emily, Samuel Davis...",Cabin Fever,Released,English,1500000.0,30351664.0,US


In [26]:
imdb_df.dropna(inplace=True)

In [65]:
#imdb_df['context2vectorize']= 'Name:'+ imdb_df['names'] + '. Genre '+ imdb_df['genre'] + '. ReleaseDate: '+ imdb_df['date_x']+ '. Overview:'+ imdb_df['overview'] + '. Crew: '+ imdb_df['crew']
imdb_df['context2vectorize']= 'Genre: '+ imdb_df['genre'] + '. ReleaseDate: '+ imdb_df['date_x']+ '. Overview:'+ imdb_df['overview']

In [66]:
imdb_df['context2vectorize'].head()

,context2vectorize
0,"Genre: Drama, Action. ReleaseDate: 03/02/2023 ..."
1,"Genre: Science Fiction, Adventure, Action. Rel..."
2,"Genre: Animation, Adventure, Family, Fantasy, ..."
3,"Genre: Animation, Comedy, Family, Adventure, F..."
4,Genre: Action. ReleaseDate: 03/17/2023 . Overv...


In [67]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [68]:
imdb_df['vector'] = imdb_df['context2vectorize'].apply(lambda sentence: model.encode(sentence))

In [69]:
imdb_df['id'] = imdb_df.index.astype(str)
imdb_df['values'] = imdb_df['vector'].tolist()
imdb_df['metadata'] = imdb_df.apply(lambda row: {'names':row['names'], 'genre':row['genre'],
                                                   'date_x':row['date_x'], 'crew':row['crew'],}, axis=1)

In [73]:
imdb_df['metadata']

,metadata
0,"{'names': 'Creed III', 'genre': 'Drama, Action..."
1,"{'names': 'Avatar: The Way of Water', 'genre':..."
2,"{'names': 'The Super Mario Bros. Movie', 'genr..."
3,"{'names': 'Mummies', 'genre': 'Animation, Come..."
4,"{'names': 'Supercell', 'genre': 'Action', 'dat..."
...,...
10173,"{'names': '20th Century Women', 'genre': 'Dram..."
10174,{'names': 'Delta Force 2: The Colombian Connec...
10175,"{'names': 'The Russia House', 'genre': 'Drama,..."
10176,"{'names': 'Darkman II: The Return of Durant', ..."


In [71]:
upsert_df = imdb_df[['id','values','metadata']]

In [72]:
upsert_df.head()

,id,values,metadata
0,0,"[-0.03598356, 0.07555795, -0.08039729, 0.01386...","{'names': 'Creed III', 'genre': 'Drama, Action..."
1,1,"[-0.10655128, -0.010176986, -0.039309576, 0.05...","{'names': 'Avatar: The Way of Water', 'genre':..."
2,2,"[-0.03470185, -0.022072706, 0.0047358763, -0.0...","{'names': 'The Super Mario Bros. Movie', 'genr..."
3,3,"[0.00544581, -0.01375277, 0.00362656, -0.03137...","{'names': 'Mummies', 'genre': 'Animation, Come..."
4,4,"[-0.012963628, 0.06071726, 0.021506475, 0.0011...","{'names': 'Supercell', 'genre': 'Action', 'dat..."


In [74]:
index = pc.Index("quickstart")
index.upsert_from_dataframe(upsert_df, namespace="quickstart")

sending upsert requests:   0%|          | 0/10052 [00:00<?, ?it/s]

{'upserted_count': 10052}

In [86]:
def query_pinecone(query, top_k):
  xc = index.query(vector = (model.encode(query)).tolist(), # python list
           top_k=10, namespace="quickstart",include_metadata=True)
  print_df = pd.DataFrame()
  temp_df = pd.DataFrame()
  for ids in xc.matches:
    print(ids.id,":", ids.score)
    temp_df= imdb_df[imdb_df['id'] == ids.id]
    print_df = pd.concat([print_df, imdb_df[imdb_df['id'] == ids.id]])
  return print_df

In [91]:
query_pinecone("List Animation genre and was released in the 2023", 10)

9877 : 0.69492352
6309 : 0.644785464
4965 : 0.634923875
3936 : 0.626339197
4363 : 0.609169304
9194 : 0.600108385
10071 : 0.597477555
1274 : 0.586599529
3940 : 0.575147867
9563 : 0.56629


,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country,context2vectorize,vector,id,values,metadata
9877,Detective Conan: Haibara Ai Monogatari ~Kuroga...,01/06/2023,0.0,Animation,We don't have an overview translated in Englis...,"Minami Takayama, Conan Edogawa (voice), Megumi...",名探偵コナン 灰原哀物語～黒鉄のミステリートレイン～,Released,Japanese,131600000.0,178359863.0,JP,Genre: Animation. ReleaseDate: 01/06/2023 . Ov...,"[-0.029857347, -0.039242275, 0.020294042, 0.00...",9877,"[-0.029857347, -0.039242275, 0.020294042, 0.00...",{'names': 'Detective Conan: Haibara Ai Monogat...
6309,Naruto 20th Anniversary - Road of Naruto,10/02/2022,80.0,"Animation, Action",Promotional video celebrating 20 years of the ...,"Junko Takeuchi, Naruto Uzumaki (voice), Noriak...",Naruto 20th Anniversary - Road of Naruto,Released,Japanese,130000000.0,765868107.0,JP,"Genre: Animation, Action. ReleaseDate: 10/02/2...","[-0.06430162, 0.0127836205, 0.027596004, -0.05...",6309,"[-0.06430162, 0.0127836205, 0.027596004, -0.05...",{'names': 'Naruto 20th Anniversary - Road of N...
4965,Irmão do Jorel - Especial Carnaval Bruttal,02/25/2022,63.0,"Animation, Comedy, Family",We don't have an overview translated in Englis...,"Andrei Duarte, Jorel's Brother (voice)",Irmão do Jorel - Especial Carnaval Bruttal,Released,Portuguese,115000000.0,355548006.2,BR,"Genre: Animation, Comedy, Family. ReleaseDate:...","[-0.023570374, -0.07871459, -0.0028695697, 0.0...",4965,"[-0.023570374, -0.07871459, -0.0028695697, 0.0...",{'names': 'Irmão do Jorel - Especial Carnaval ...
3936,Princess Principal Crown Handler: Chapter 3,04/07/2023,0.0,"Animation, Action, Adventure, Drama, Mystery",The third out of six movie sequels to the TV s...,"Aoi Koga, Ange (voice), Akira Sekine, Princess...",プリンセス・プリンシパル Crown Handler 第3章,Released,Japanese,231940000.0,178359863.0,JP,"Genre: Animation, Action, Adventure, Drama, My...","[-0.042868387, -0.10407827, -0.0002727895, 0.0...",3936,"[-0.042868387, -0.10407827, -0.0002727895, 0.0...",{'names': 'Princess Principal Crown Handler: C...
4363,The Animatrix,05/09/2003,72.0,"Animation, Science Fiction",Straight from the creators of the groundbreaki...,"Keanu Reeves, Thomas ""Neo"" Anderson (voice), C...",The Animatrix,Released,English,5000000.0,121345628.6,AU,"Genre: Animation, Science Fiction. ReleaseDate...","[-0.053557202, -0.0043882187, -0.08185281, -0....",4363,"[-0.053557202, -0.0043882187, -0.08185281, -0....","{'names': 'The Animatrix', 'genre': 'Animation..."
9194,K: Seven Stories Movie 1 - R:B - Blaze,07/07/2018,67.0,"Animation, Drama, Action",We don't have an overview translated in Englis...,"Daisuke Namikawa, Yashiro Isana, Daisuke Ono, ...",K Seven Stories R:b ～BLAZE～,Released,Japanese,120040000.0,549268906.0,JP,"Genre: Animation, Drama, Action. ReleaseDate: ...","[0.013331756, -0.08869198, 0.0018090522, 0.018...",9194,"[0.013331756, -0.08869198, 0.0018090522, 0.018...",{'names': 'K: Seven Stories Movie 1 - R:B - Bl...
10071,Making 'Toy Story',12/02/1995,70.0,Documentary,Documentary of the making of the groundbreakin...,"John Lasseter, Self, Ed Catmull, Self, Andrew ...",Making 'Toy Story',Released,English,100140523.8,625891229.0,US,Genre: Documentary. ReleaseDate: 12/02/1995 . ...,"[-0.09967347, -0.034967504, 0.012988104, 0.015...",10071,"[-0.09967347, -0.034967504, 0.012988104, 0.015...","{'names': 'Making 'Toy Story'', 'genre': 'Docu..."
1274,Princess Principal Crown Handler: Chapter 2,09/23/2021,73.0,"Animation, Action, Adventure, Drama, Mystery",The second out of six movie sequels to the TV ...,"Aoi Koga, Ange (voice), Akira Sekine, Princess...",プリンセス・プリンシパル Crown Handler 第2章,Released,Japanese,109000000.0,692214960.4,JP,"Genre: Animation, Action, Adventure, Drama, My...","[-0.02645821, -0.0771603, 0.027775422, 0.04712...",1274,"[-0.02645821, -0.0771603, 0.027775422, 0.04712...",{'names': 'Princess Principal Crown Handler: C...
3940,Kuroko's Basketball - Movie: Winter Cup - Cros...,12/03/2016,75.0,Animation,The third of 

In [90]:
query_pinecone("List Action genre and with released in 2023 ", 10)

5642 : 0.666301966
4647 : 0.655798912
1293 : 0.594980717
4614 : 0.583581865
3660 : 0.578115
9194 : 0.577294707
3936 : 0.573407948
9563 : 0.557729363
1274 : 0.557339847
8222 : 0.552299142


,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country,context2vectorize,vector,id,values,metadata
5642,Garota da Moto,09/23/2021,47.0,Action,We don't have an overview translated in Englis...,"Maria Casadevall, Joana, Kevin Vechiatto, Nico...",Garota da Moto,Released,Portuguese,129500000.0,847860141.2,BR,Genre: Action. ReleaseDate: 09/23/2021 . Overv...,"[-0.0048249546, -0.08295113, -0.009472517, 0.0...",5642,"[-0.0048249546, -0.08295113, -0.009472517, 0.0...","{'names': 'Garota da Moto', 'genre': 'Action',..."
4647,Super Bodyguard,04/22/2021,60.0,"Action, Drama",We don't have an overview translated in Englis...,"Xie Mengwei, Ye Kai, Hao Shaowen, Tony, Cui Zh...",超神保镖,Released,Chinese,92200000.0,254504167.2,CN,"Genre: Action, Drama. ReleaseDate: 04/22/2021 ...","[0.037994817, -0.10099548, 0.0040767896, 0.028...",4647,"[0.037994817, -0.10099548, 0.0040767896, 0.028...","{'names': 'Super Bodyguard', 'genre': 'Action,..."
1293,Adanis: Kutsal Kavga,03/11/2022,51.0,"Drama, Action, Adventure",We don't have an overview translated in Englis...,"Esra Bilgiç, Cemre, İsmail Filiz, İlhan, Baki ...",Adanis: Kutsal Kavga,Released,Turkish,3740000.0,568828.0,TR,"Genre: Drama, Action, Adventure. ReleaseDate: ...","[0.029078005, -0.06581534, 0.014406273, 0.0229...",1293,"[0.029078005, -0.06581534, 0.014406273, 0.0229...","{'names': 'Adanis: Kutsal Kavga', 'genre': 'Dr..."
4614,Ahimsa,04/07/2023,0.0,"Romance, Action, Drama",We don't have an overview translated in Englis...,"Abhiram Daggubati, Raghu, Geethika Tiwary, Aha...",టీజ‌ర్‌,Released,Telugu,223940000.0,175269998.8,IN,"Genre: Romance, Action, Drama. ReleaseDate: 04...","[0.0099479025, -0.09320628, 0.045923725, 0.056...",4614,"[0.0099479025, -0.09320628, 0.045923725, 0.056...","{'names': 'Ahimsa', 'genre': 'Romance, Action,..."
3660,Meg 2: The Trench,08/04/2023,0.0,"Science Fiction, Action, Horror, Thriller",We don't have an overview translated in Englis...,"Jason Statham, Jonas Taylor, Shuya Sophia Cai,...",Meg 2: The Trench,Post Production,English,231940000.0,178359863.0,US,"Genre: Science Fiction, Action, Horror, Thrill...","[-0.00069101667, -0.091042995, -0.000510626, 0...",3660,"[-0.00069101667, -0.091042995, -0.000510626, 0...","{'names': 'Meg 2: The Trench', 'genre': 'Scien..."
9194,K: Seven Stories Movie 1 - R:B - Blaze,07/07/2018,67.0,"Animation, Drama, Action",We don't have an overview translated in Englis...,"Daisuke Namikawa, Yashiro Isana, Daisuke Ono, ...",K Seven Stories R:b ～BLAZE～,Released,Japanese,120040000.0,549268906.0,JP,"Genre: Animation, Drama, Action. ReleaseDate: ...","[0.013331756, -0.08869198, 0.0018090522, 0.018...",9194,"[0.013331756, -0.08869198, 0.0018090522, 0.018...",{'names': 'K: Seven Stories Movie 1 - R:B - Bl...
3936,Princess Principal Crown Handler: Chapter 3,04/07/2023,0.0,"Animation, Action, Adventure, Drama, Mystery",The third out of six movie sequels to the TV s...,"Aoi Koga, Ange (voice), Akira Sekine, Princess...",プリンセス・プリンシパル Crown Handler 第3章,Released,Japanese,231940000.0,178359863.0,JP,"Genre: Animation, Action, Adventure, Drama, My...","[-0.042868387, -0.10407827, -0.0002727895, 0.0...",3936,"[-0.042868387, -0.10407827, -0.0002727895, 0.0...",{'names': 'Princess Principal Crown Handler: C...
9563,K: Seven Stories Movie 5 - Memory of Red - Burn,11/03/2018,65.0,"Animation, Action, Fantasy",We don't have an overview translated in Englis...,"Kenjiro Tsuda, Mikoto Suou, Yuki Kaji, Tatara ...",K Seven Stories メモリー・オブ・レッド ~BURN~,Released,Japanese,141280000.0,608857282.0,JP,"Genre: Animation, Action, Fantasy. ReleaseDate...","[0.005066107, -0.072665796, 0.0006476646, 0.02...",9563,"[0.005066107, -0.072665796, 0.0006476646, 0.02...",{'names': 'K: Seven Stories Movie 5 - Memory o...
1274,Princess Principal Crown Handler: Chapter 2,09/23/2021,73.0,"Animation, Action, Adventure, Drama, Mystery",The second out of six movie sequels to the TV ...,"Aoi Koga, Ange (voice), Akira Sekine, Princess...",プリンセス・プリンシパル Crown Hand